In [4]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install plotly.express

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime, date
dt_string = datetime.now().strftime('%y%m%d')
pd.set_option('display.max_columns', 500)

In [2]:
cd_name = "221207-0943-cf11_cd_21-output.xlsx"
cd_cf11 = pd.read_excel(f'{cd_name}', sheet_name='DB', dtype=str)
cd_cf11['origen'] = 'CD'

In [3]:
tienda_name = "221205_post.xlsx"
tienda_cf11 = pd.read_excel(f'{tienda_name}', sheet_name='DB', dtype=str)
tienda_cf11['origen'] = 'TIENDA'

In [4]:
cd_to_del = ['indice_cf11','prd_upc', 'transportadora_nuevo', 'reporte_a_contabilidad', 'movimiento_contable',
        'tranf_electro_factura', 'f3', 'f4', 'f5', 'f11_nuevo', 'f12',
       'nota', 'ro', 'gco_dup', 'dupmc', 'error_ru', 'gco_dupall']
cd_cf11.drop(cd_to_del, axis=1, inplace=True)

In [5]:
tienda_to_del = ['indice_c11t', 'ean', 'propietario', 'f']
tienda_cf11.drop(tienda_to_del, axis=1, inplace=True)

In [6]:
tienda_cf11.rename(columns={'folio_servicio_tecnico':'n_folio', 'estado_servicio_tecnico':'estado_f11',
       'costo_promedio':'costo_total', 'tipo_servicio_f11': 'xservicio', 'motivo_cierre':'status_final', 'cantidad_f11':'qproducto'}, inplace=True)

In [7]:
con = pd.concat([cd_cf11, tienda_cf11])
con['partition_date'] = date.today()
con['costo_total'] = pd.to_numeric(con['costo_total'])
con['status_final'] = con.status_final.str.capitalize()

In [8]:
con['Comentario GCO'].unique()

array(['Coincidencia exacta F4+UPC+QTY', 'Coincidencia exacta F3+UPC+QTY',
       'Coincidencia exacta (F12|F11)', 'No coincide (F12|F11)',
       'No coincide F5|UPC|Qty', nan,
       'Duplicidad verificada F12+UPC+Cantidad',
       'Registro no duplicado F12+UPC+Cantidad',
       'Coincidencia exacta F5+UPC+QTY', 'No coincide F3|UPC|QTY',
       'F11 nuevo en estado despachado',
       'Duplicidad verificada F11+UPC+Cantidad', 'No coincide F4|UPC|QTY',
       'Coincidencia (F12|F11) + Registro duplicado en DB',
       'Coincidencia F4+UPC+QTY + Registro duplicado en DB',
       'Registro duplicado F12 + UPC + Cantidad',
       'Cantidad de las F11s de un F5 > cantidad del F5',
       'Registro anulado', 'Registro sin nro. de F4',
       'Coincidencia F3+UPC+QTY + Registro duplicado en DB',
       'Coincidencia exacta (F11|F3)+UPC+QTY',
       'No coincide (F11|F3)+UPC+QTY',
       'Coincidencia Exacta (F11|DV)+UPC+QTY',
       'Coincidencia exacta (F11|F4)+UPC+QTY', 'No coincide (F11

In [8]:
con_nov = [
       'Coincidencia (F12|F11) - Se requiere F12|UPC|QTY para validar duplicidad',
       'F11 Registro sin nro. de f11 nuevo', 
       'No coincide F3|UPC|QTY', 'No coincide (F12|F11)',
       'No coincide F5|UPC|Qty', 'Registro sin nro. de F4',
       'F11 nuevo no encontrado',
       'Recibido con fecha anterior al 14/01/2022', 
       'Recibido con fecha posterior al 14/01/2022', 
       'Registro duplicado F12 + UPC + Cantidad',
       'Coincidencia F4+UPC+QTY + Registro duplicado en DB',
       'Coincidencia (F12|F11) + Registro duplicado en DB',
       'Registro con estado diferente a recibido',
       'Cantidad de los F11s de un F4 > cantidad del F4',
       'No coincide F4|UPC|QTY',
       'Coincidencia F3+UPC+QTY + Registro duplicado en DB',
       'No coincide (F11|F4)+UPC+QTY', 'No coincide (F11|F3)+UPC+QTY',
       'Registro sin nro. de F5',
       'Registro con año de reserva diferente a 2022',
       'La cantidad sumada de los F11s de un F5 es mayor que la cantidad del F5',
       'Registro duplicado F+UPC+Cantidad',
       'Registro con año de confirmación diferente a 2022',
       'Cantidad de los F11s de un F3 > cantidad del F3',
       'Registro anulado','Coincidencia F5+UPC+QTY - Se requiere F12|UPC|QTY para validar duplicidad',
       'Cantidad de las F11s de un F5 > cantidad del F5','Error de clasificacion - Duplicidad F12+UPC+Cantidad', 'F11 nuevo encontrado (diferente estado)', 'Error de clasificacion - duplicidad F11 + upc +cantidad','No coincide (F11|DV)',
       'Coincidencia Exacta (F11|DV) QTY diferente',
       'Coincidencia F4+UPC+QTY - Se requiere F12|UPC|QTY para validar duplicidad', 'Coincidencia F3+UPC+QTY - Se requiere F12|UPC|QTY para validar duplicidad','Registro no duplicado F12+UPC+Cantidad']

sin_nov = ['Coincidencia exacta F4+UPC+QTY', 'Coincidencia exacta F3+UPC+QTY',
'Coincidencia exacta (F12|F11)', 'F11 nuevo en estado despachado',
'Coincidencia exacta F5+UPC+QTY',
'Coincidencia exacta (F11|F3)+UPC+QTY', 'Coincidencia exacta (F11|F4)+UPC+QTY',
'Duplicidad verificada F12+UPC+Cantidad', 'Duplicidad verificada F11+UPC+Cantidad',
 'Error f11', 'Registro de f11 nuevo encontrado','Coincidencia Exacta (F11|DV)+UPC+QTY',]

con.loc[con['Comentario GCO'].isin(con_nov), 'rev'] = 'Con novedad'
con.loc[con['Comentario GCO'].isin(sin_nov), 'rev'] = 'Sin novedad'
con['rev'] = con['rev'].fillna('Pendiente')


In [9]:
con.loc[con['rev'].isna(), 'Comentario GCO'].unique()

array([], dtype=object)

In [10]:
con.loc[(con['origen']=='TIENDA') & (con['Comentario GCO'].isna()), 'rev'] = 'Cierre en tienda'

In [11]:
# revision sin novedad
con.loc[(con['rev'] == 'Pendiente') & (con['Comentario GCO'].notna())]['Comentario GCO'].unique() 

array([], dtype=object)

In [12]:
con.loc[(con['rev'] == 'Con novedad') & (con['origen'] == 'CD'),'Comentario GCO'].unique()

array(['No coincide (F12|F11)', 'No coincide F5|UPC|Qty',
       'Registro no duplicado F12+UPC+Cantidad', 'No coincide F3|UPC|QTY',
       'No coincide F4|UPC|QTY',
       'Coincidencia (F12|F11) + Registro duplicado en DB',
       'Coincidencia F4+UPC+QTY + Registro duplicado en DB',
       'Registro duplicado F12 + UPC + Cantidad',
       'Cantidad de las F11s de un F5 > cantidad del F5',
       'Registro anulado', 'Registro sin nro. de F4',
       'Coincidencia F3+UPC+QTY + Registro duplicado en DB'], dtype=object)

In [13]:
con.columns

Index(['nfolio', 'qproducto', 'xobservacion', 'xservicio', 'costo_total',
       'estado_f11', 'status_final', 'sku', 'GCO', 'Comentario GCO', 'checked',
       'fnandup', 'origen', 'n_folio', 'local_envio',
       'sku_(numero_de_producto)', 'F11', 'SKU ORIGINAL',
       'CANT. UND GENERADAS', 'partition_date', 'rev'],
      dtype='object')

In [14]:
# CD
cd_fi = [ 'Escalamiento li',  'Ingreso con sku trocado', 'Pendiente revisar', 'Ingresa con f12 trocado']
cd_error = ['Error en creacion de f11', 'Con nuevo f11 en estado despachado', 'Error de cierre', 'Error en creacia3n de f11']
con.loc[con.status_final.isin(cd_fi), 'status_final'] = 'Falta infomación CD'
con.loc[con.status_final.isin(cd_error), 'status_final'] = 'Error cierre/creación F11'
duplicados = ['Cierre x duplicidad f11+upc+qty', 'Cierre x duplicidad f12+upc+qty']
con.loc[con.status_final.isin(duplicados), 'status_final'] = 'Duplicados'
con.loc[con.status_final=='Cierre x producto guardado despues de inventario', 'status_final'] = 'Guardado después de inv'
con.loc[con.status_final=='Cierre x producto guardado antes de inventario', 'status_final'] = 'Guardado antes de inv'


con.loc[con.status_final=='Cierre x f3 devuelto a proveedor', 'status_final'] = 'F3 devuelto a proveedor'
con.loc[con.status_final=='Cierre x f4 cobrado a terceros', 'status_final'] = 'F4 cobrado a terceros'


In [15]:
# Tienda
tienda_ct = ['Sin informacion - tienda', 'Duplicado con ajuste en tienda',
       'Entregado en tienda', 'Cambio producto x nuevo', 'Dv','Ajuste inventarios', 
       'Entregado a cliente', 'Validar cierre con chile', 'F11 concesion', 'Nuevo f11']
con.loc[con.status_final.isin(tienda_ct), 'status_final'] = 'Cierre en tienda'
con.loc[con.status_final=='F3', 'status_final'] = 'F3 devuelto a proveedor'
con.loc[con.status_final=='F4 en tienda', 'status_final'] = 'F4 de merma'
con.loc[con.status_final=='F4', 'status_final'] = 'F4 de merma'

In [16]:
con['status_final'].replace('Cierre por producto guardado despues de inventario','Guardado despues de inv', inplace= True)
con['status_final'].replace('Cierre por f4 cobrado a terceros','F4 cobrado a terceros', inplace= True)
con['status_final'].replace('Cierre por f3 devuelto a proveedor','F3', inplace= True)
con['status_final'].replace('F4 de merma','F4 merma', inplace= True)
con['status_final'].replace('F3 devuelto a proveedor','F3', inplace= True)

In [17]:
con_total = con.groupby('status_final')['costo_total'].sum().sort_values(ascending=False).reset_index()

In [18]:
lista = con_total['status_final'][0:6].tolist()

In [19]:
lista

['F3',
 'Guardado despues de inv',
 'F4 cobrado a terceros',
 'Cierre en tienda',
 'F4 merma',
 'Duplicados']

In [20]:
con_graf = con_total.loc[con_total['status_final'].isin(lista)]

In [26]:
con_graf['percent'] = round((con_graf['costo_total'] / con_graf['costo_total'].sum()) * 100)
con_graf['percent'] = con_graf['percent'].astype(str)
con_graf['str_costo_total'] = round(con_graf['costo_total']/1000000000,1)
con_graf['str_costo_total'] = con_graf['str_costo_total'].astype(str)

C:\Users\jlrodriguezc\AppData\Local\Temp\ipykernel_13100\2210615400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con_graf['percent'] = round((con_graf['costo_total'] / con_graf['costo_total'].sum()) * 100)
C:\Users\jlrodriguezc\AppData\Local\Temp\ipykernel_13100\2210615400.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con_graf['percent'] = con_graf['percent'].astype(str)
C:\Users\jlrodriguezc\AppData\Local\Temp\ipykernel_13100\2210615400.py:3: SettingWithCopyWarning: 
A value is trying to be se

In [33]:
con_graf['concat'] = con_graf['str_costo_total'] + 'G' + ' ' + '(' + con_graf['percent'] + '%' + ')'
con_graf

C:\Users\jlrodriguezc\AppData\Local\Temp\ipykernel_13100\3008671593.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,status_final,costo_total,percent,str_costo_total,concat
0,F3,6.608643e+09,33.0,6.6,6.6G (33.0%)
1,Guardado despues de inv,4.841510e+09,24.0,4.8,4.8G (24.0%)
2,F4 cobrado a terceros,3.566596e+09,18.0,3.6,3.6G (18.0%)
3,Cierre en tienda,3.524980e+09,18.0,3.5,3.5G (18.0%)
4,F4 merma,1.140926e+09,6.0,1.1,1.1G (6.0%)
5,Duplicados,4.367837e+08,2.0,0.4,0.4G (2.0%)


In [28]:
total_f11s = con_total['costo_total'].sum()

In [45]:
fig_con_total = px.bar(con_graf, x ='status_final', y='costo_total', title=f"Costo total de F11s según motivos de cierre - $ {total_f11s/1e6:,.0f}M", labels={ 'costo_total':'Total costo promedio', 'status_final':'Motivo de cierre', 'GRUPO':'Local'}, text='concat') #category_orders=co ) 
#fig_con_total.update_layout(legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.1))
fig_con_total.update_layout(xaxis_tickangle = 30)
fig_con_total.write_image(f'C:/Users/jlrodriguezc/Proyectos/Slides_f/output/{dt_string}_con_total.svg',width=1050, height=300, engine='orca') # 820 , 350 
fig_con_total.show()

In [73]:
con.loc[(con.origen =='CD') & (con['rev'] == 'Pendiente'), 'status_final'] = 'Falta infomación CD'

In [74]:
con_cd = con.loc[con.origen =='CD'].groupby(['status_final', 'rev'], sort=False)['costo_total'].sum().sort_values(ascending=False).reset_index()

In [75]:
motivos = con_cd.status_final.unique()
gb_motivos_cd = con_cd.groupby('status_final')['costo_total'].sum().reset_index()
gb_motivos_cd.set_index('status_final', inplace=True)
for motivo in motivos:
    con_cd.loc[con_cd['status_final']==motivo, 'status_final'] = f'{motivo} - ${round(gb_motivos_cd.loc[motivo, "costo_total"]/1e6):,.0f}M'

In [76]:
revs = con_cd.rev.unique()
gb_rev_cd = con_cd.groupby('rev')['costo_total'].sum().reset_index()
gb_rev_cd.set_index('rev', inplace=True)
for rev in revs:
    con_cd.loc[con_cd['rev']==rev, 'rev'] = f'{rev} - ${round(gb_rev_cd.loc[rev, "costo_total"]/1e6):,.0f}M'

In [77]:
revs =  [con_cd.loc[con_cd['rev'].str.contains(r'Sin novedad'), 'rev'].unique()[0], con_cd.loc[con_cd['rev'].str.contains(r'Con novedad'), 'rev'].unique()[0], con_cd.loc[con_cd['rev'].str.contains(r'Pendiente'), 'rev'].unique()[0]]
co_cd = {'rev': revs, 'status_final':con_cd.status_final.unique()}

In [78]:
total_cd= con_cd['costo_total'].sum()

In [79]:
fig_con_total = px.bar(con_cd, orientation='h', y ='status_final', x='costo_total', color='rev', category_orders=co_cd, title= f"Motivos de cierres de F11 CD 2022 - $ {total_cd/1e6:,.0f}M", 
labels={ 'costo_total':'Total costo promedio', 'status_final':'Motivo de cierre', 'rev':'Estado revisión'}, text='costo_total',text_auto='.2s', 
color_discrete_sequence=[ 'rgb(115, 175, 72)','rgb(204, 80, 62)','rgb(179, 179, 179)' ] ) 
fig_con_total.update_layout(legend=dict(yanchor="bottom", y=0.1, xanchor="right", x=0.9))
fig_con_total.update_layout(margin_r=20, margin_t=60)
fig_con_total.update_xaxes(range = [0,con_cd['costo_total'].max() + con_cd['costo_total'].max() * 0.2], constrain = 'domain')
fig_con_total.write_image(f'C:/Users/maperezr/proyectos/slides-fs/output/{dt_string}_con_cd.svg',width=1000, height=400, engine='orca') # 820 , 350 
fig_con_total.show()

In [80]:
con_tienda = con.loc[con.origen =='TIENDA'].groupby(['status_final', 'rev'])['costo_total'].sum().sort_values(ascending=False).reset_index()

In [81]:
con_tienda

,status_final,rev,costo_total
0,F3,Sin novedad,4.752565e+09
1,Cierre en tienda,Cierre en tienda,2.106981e+09
2,Cierre en tienda,Sin novedad,1.388588e+09
3,F4 merma,Sin novedad,3.674360e+08
4,F5,Sin novedad,1.783761e+08
5,F3,Con novedad,3.721489e+07
6,Cierre en tienda,Con novedad,2.941116e+07
7,F4 merma,Con novedad,2.581822e+07
8,F5,Con novedad,1.429123e+06


In [82]:
motivos = con_tienda.status_final.unique()
gb_motivos_tienda = con_tienda.groupby('status_final')['costo_total'].sum().reset_index()
gb_motivos_tienda.set_index('status_final', inplace=True)
for motivo in motivos:
    con_tienda.loc[con_tienda['status_final']==motivo, 'status_final'] = f'{motivo} - ${round(gb_motivos_tienda.loc[motivo, "costo_total"]/1e6):,.0f}M'

In [83]:
revs = con_tienda.rev.unique()
gb_rev_tienda = con_tienda.groupby('rev')['costo_total'].sum().reset_index()
gb_rev_tienda.set_index('rev', inplace=True)
for rev in revs:
    con_tienda.loc[con_tienda['rev']==rev, 'rev'] = f'{rev} - ${round(gb_rev_tienda.loc[rev, "costo_total"]/1e6):,.0f}M'

In [84]:
revs

array(['Sin novedad', 'Cierre en tienda', 'Con novedad'], dtype=object)

In [85]:
con_tienda.loc[con_tienda['rev'].str.contains(r'Sin novedad'), 'rev'].unique()[0], con_tienda.loc[con_tienda['rev'].str.contains(r'Con novedad'), 'rev'].unique()[0], con_tienda.loc[con_tienda['rev'].str.contains(r'Cierre en tienda'), 'rev'].unique()[0]

('Sin novedad - $6,687M', 'Con novedad - $94M', 'Cierre en tienda - $2,107M')

In [86]:
revs_tienda =  [con_tienda.loc[con_tienda['rev'].str.contains(r'Sin novedad'), 'rev'].unique()[0], con_tienda.loc[con_tienda['rev'].str.contains(r'Con novedad'), 'rev'].unique()[0], con_tienda.loc[con_tienda['rev'].str.contains(r'Cierre en tienda'), 'rev'].unique()[0]]
co_tienda = {'rev':revs_tienda, 'status_final':con_tienda.status_final.unique()}

In [87]:
total_tienda = con_tienda['costo_total'].sum()

In [88]:
fig_con_total = px.bar(con_tienda, orientation='h', y ='status_final', x='costo_total', title= f"Motivos de cierres de F11 Tienda 2022 - $ {total_tienda/1e6:,.0f}M", 
labels={ 'costo_total':'Total costo promedio', 'status_final':'Motivo de cierre', 'rev':'Estado revisión'}, text='costo_total',text_auto='.2s', 
color_discrete_sequence=[ 'rgb(115, 175, 72)','rgb(204, 80, 62)','rgb(179, 179, 179)' ], color ='rev', category_orders=co_tienda ) 
fig_con_total.update_layout(legend=dict(yanchor="bottom", y=0.1, xanchor="right", x=0.9))
fig_con_total.update_layout(margin_r=20, margin_t=60)
fig_con_total.update_xaxes(range = [0,con_tienda['costo_total'].max() + con_tienda['costo_total'].max() * 0.2], constrain = 'domain')
fig_con_total.write_image(f'C:/Users/maperezr/proyectos/slides-fs/output/{dt_string}_con_tienda.svg',width=900, height=400, engine='orca') # 820 , 350 
fig_con_total.show()

In [89]:
con.head()

,nfolio,qproducto,xobservacion,xservicio,costo_total,estado_f11,status_final,sku,GCO,Comentario GCO,checked,fnandup,origen,n_folio,local_envio,sku_(numero_de_producto),F11,SKU ORIGINAL,CANT. UND GENERADAS,partition_date,rev
0,1110454448,2,se cierra 1110454448 asociado a 12630103969 si...,retiro f12,29334.0,entrega total,F4 merma,9181931,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-07,Sin novedad
1,1110454448,2,12630103969,retiro f12,29334.0,entrega total,F4 merma,9181933,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-07,Sin novedad
2,1110484339,1,12607042673se cierra 1110484339 asociado a 126...,retiro f12,1210950.0,entrega total,F4 merma,3225964,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-07,Sin novedad
3,1110544224,1,12620043022se cierra 1110544224 asociado a 126...,retiro f12,157999.0,entrega total,F4 merma,9988778,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-07,Sin novedad
4,1110580802,1,12619745159se cierra 1110580802 asociado a 126...,retiro f12,2544730.0,entrega total,F4 merma,3394545,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-07,Sin novedad


In [90]:
con.to_excel('consolidado_cf11.xlsx', index=False)